Importing libraries

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Loading our raw dataset

In [122]:
data_v0 = pd.read_csv("WDICSV.csv")

Inspecting the raw data

In [123]:
data_v0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397936 entries, 0 to 397935
Data columns (total 68 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Country Name    397936 non-null  object 
 1   Country Code    397936 non-null  object 
 2   Indicator Name  397936 non-null  object 
 3   Indicator Code  397936 non-null  object 
 4   1960            37854 non-null   float64
 5   1961            42942 non-null   float64
 6   1962            44259 non-null   float64
 7   1963            45206 non-null   float64
 8   1964            45714 non-null   float64
 9   1965            47808 non-null   float64
 10  1966            47816 non-null   float64
 11  1967            48461 non-null   float64
 12  1968            49074 non-null   float64
 13  1969            49838 non-null   float64
 14  1970            73611 non-null   float64
 15  1971            79889 non-null   float64
 16  1972            81646 non-null   float64
 17  1973      

So, we got Country name, Indicators, the separate years as columns. The data is not in very good state, have so much of nulls and the structure of the data is not very dashboard friendly.

First, we need to filter out on relevant indicators pertaining to our objective. We are tracking UN SDG 3 targets and as per literature... most important indicators for this goal are:

1. Health Expenditure (% of GDP)
2. Life Expectancy (years)
3. Infant Mortality Rate (per 1000 live births)
4. Maternal Mortality Rate (per 100000 live births)
5. Neonatal Mortality Rate (per 1000 live births)
6. Under 5 Mortality Rate (per 1000 live births)
7. TB incidents (per 100k people)
8. Undernourishment prevalance (% of population)

Only these indicators will be chosen from the data

Upon inspecting the metadata, we found the indicator code for these indicators. 

1. SP.DYN.IMRT.IN	-	IMR
2. SH.STA.MMRT	    -   MMR
3. SH.DYN.NMRT	    -	NMR
4. SN.ITK.DEFC.ZS	-	UND
5. SP.DYN.LE00.IN	-	LE
6. SH.XPD.CHEX.GD.ZS	-	CHE
7. SH.DYN.MORT	-	U5 Mortality
8. SH.TBS.INCD	-	Tuberculosis


In [124]:
#filtering out these indicators

indicator_list = ['SP.DYN.IMRT.IN', 'SH.STA.MMRT', 'SH.DYN.NMRT','SN.ITK.DEFC.ZS','SP.DYN.LE00.IN','SH.XPD.CHEX.GD.ZS','SH.DYN.MORT','SH.TBS.INCD']
df0 = data_v0[data_v0['Indicator Code'].isin(indicator_list)]



In [125]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2128 entries, 290 to 397577
Data columns (total 68 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    2128 non-null   object 
 1   Country Code    2128 non-null   object 
 2   Indicator Name  2128 non-null   object 
 3   Indicator Code  2128 non-null   object 
 4   1960            546 non-null    float64
 5   1961            549 non-null    float64
 6   1962            553 non-null    float64
 7   1963            562 non-null    float64
 8   1964            575 non-null    float64
 9   1965            586 non-null    float64
 10  1966            591 non-null    float64
 11  1967            603 non-null    float64
 12  1968            623 non-null    float64
 13  1969            648 non-null    float64
 14  1970            654 non-null    float64
 15  1971            669 non-null    float64
 16  1972            679 non-null    float64
 17  1973            684 non-null 

In [126]:
df0['Indicator Name'].unique()

array(['Current health expenditure (% of GDP)',
       'Incidence of tuberculosis (per 100,000 people)',
       'Life expectancy at birth, total (years)',
       'Maternal mortality ratio (modeled estimate, per 100,000 live births)',
       'Mortality rate, infant (per 1,000 live births)',
       'Mortality rate, neonatal (per 1,000 live births)',
       'Mortality rate, under-5 (per 1,000 live births)',
       'Prevalence of undernourishment (% of population)'], dtype=object)

Indicator filter is applied. Now we have to decide upon the time range. As my analysis will be focussed for 21st century. I will only take values since 1999. (1999 to be kept for comparison purposes.) 

In [127]:
base_cols = ['Country Name', 'Country Code', 'Indicator Name']
year_cols = [col for col in df0.columns if col.isdigit() and int(col) >= 1999]
cols = base_cols + year_cols
df1 = df0[cols]


In [128]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2128 entries, 290 to 397577
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    2128 non-null   object 
 1   Country Code    2128 non-null   object 
 2   Indicator Name  2128 non-null   object 
 3   1999            990 non-null    float64
 4   2000            1676 non-null   float64
 5   2001            1894 non-null   float64
 6   2002            1896 non-null   float64
 7   2003            1898 non-null   float64
 8   2004            1898 non-null   float64
 9   2005            1901 non-null   float64
 10  2006            1900 non-null   float64
 11  2007            1900 non-null   float64
 12  2008            1900 non-null   float64
 13  2009            1900 non-null   float64
 14  2010            1903 non-null   float64
 15  2011            1905 non-null   float64
 16  2012            1904 non-null   float64
 17  2013            1904 non-null

Now, I will rename indicators to much simpler names but make sure that they do not loose meaning

In [129]:
df1['Indicator Name'].unique()

array(['Current health expenditure (% of GDP)',
       'Incidence of tuberculosis (per 100,000 people)',
       'Life expectancy at birth, total (years)',
       'Maternal mortality ratio (modeled estimate, per 100,000 live births)',
       'Mortality rate, infant (per 1,000 live births)',
       'Mortality rate, neonatal (per 1,000 live births)',
       'Mortality rate, under-5 (per 1,000 live births)',
       'Prevalence of undernourishment (% of population)'], dtype=object)

In [130]:
indicator_rename_map = {
    'Current health expenditure (% of GDP)': 'health_exp',
    'Incidence of tuberculosis (per 100,000 people)': 'inci_tuberc',
    'Life expectancy at birth, total (years)': 'life_expect',
    'Maternal mortality ratio (modeled estimate, per 100,000 live births)': 'maternal_mortality',
    'Mortality rate, infant (per 1,000 live births)': 'infant_mortality',
    'Mortality rate, neonatal (per 1,000 live births)': 'neonatal_mortality',
    'Mortality rate, under-5 (per 1,000 live births)': 'under_5_mortality',
    'Prevalence of undernourishment (% of population)': 'prev_undernourishment'
}

In [131]:
df1['Indicator Name'] = df1['Indicator Name'].replace(indicator_rename_map)


C:\Users\apraj\AppData\Local\Temp/ipykernel_12568/3144622994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Indicator Name'] = df1['Indicator Name'].replace(indicator_rename_map)


In [132]:
df1['Indicator Name'].unique()

array(['health_exp', 'inci_tuberc', 'life_expect', 'maternal_mortality',
       'infant_mortality', 'neonatal_mortality', 'under_5_mortality',
       'prev_undernourishment'], dtype=object)

Now, I will convert this data into more dashboard friendly version. Pivoting the data, so that each indicator get its own column.

In [133]:
df_long = df1.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name'],
    var_name='year',
    value_name='value'
)

df_pivot = df_long.pivot_table(
    index=['Country Name', 'Country Code', 'year'],
    columns='Indicator Name',
    values='value'
).reset_index()

In [134]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6585 entries, 0 to 6584
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country Name           6585 non-null   object 
 1   Country Code           6585 non-null   object 
 2   year                   6585 non-null   object 
 3   health_exp             5167 non-null   float64
 4   inci_tuberc            5429 non-null   float64
 5   infant_mortality       5856 non-null   float64
 6   life_expect            6190 non-null   float64
 7   maternal_mortality     4893 non-null   float64
 8   neonatal_mortality     5856 non-null   float64
 9   prev_undernourishment  4805 non-null   float64
 10  under_5_mortality      5856 non-null   float64
dtypes: float64(8), object(3)
memory usage: 566.0+ KB


So this completes our first step, studying the raw data, choosing suitable indicators, time range, and transforming the structure of data

In [135]:
#Exporting this data

df_pivot.to_csv("stage_0.csv", index = False)

Importing transformed dataset 

In [136]:
df = pd.read_csv("stage_0.csv")

Basic info about dataset

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6585 entries, 0 to 6584
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country Name           6585 non-null   object 
 1   Country Code           6585 non-null   object 
 2   year                   6585 non-null   int64  
 3   health_exp             5167 non-null   float64
 4   inci_tuberc            5429 non-null   float64
 5   infant_mortality       5856 non-null   float64
 6   life_expect            6190 non-null   float64
 7   maternal_mortality     4893 non-null   float64
 8   neonatal_mortality     5856 non-null   float64
 9   prev_undernourishment  4805 non-null   float64
 10  under_5_mortality      5856 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 566.0+ KB


In [138]:
df = df.rename(columns={
    'Country Name': 'country',
    'Country Code': 'code',
})

In [139]:
df.describe()

,year,health_exp,inci_tuberc,infant_mortality,life_expect,maternal_mortality,neonatal_mortality,prev_undernourishment,under_5_mortality
count,6585.000000,5167.000000,5429.000000,5856.000000,6190.000000,4893.000000,5856.000000,4805.000000,5856.000000
mean,2010.938041,6.200575,126.391393,28.966417,70.066527,200.863274,16.410876,11.102426,40.399726
std,7.172526,2.790524,179.738222,25.493792,8.667176,266.881053,12.624944,10.534564,40.898652
min,1999.000000,1.107200,0.000000,1.300000,41.957000,1.000000,0.700000,2.500000,1.500000
25%,2005.000000,4.246770,14.000000,8.400000,64.439894,18.000000,5.200000,2.631544,9.900000
50%,2011.000000,5.443990,51.000000,20.100000,71.839268,74.000000,12.700000,7.100000,23.606879
75%,2017.000000,7.790887,177.000000,44.400000,76.612000,297.000000,25.700000,15.965738,60.100000
max,2023.000000,24.283052,1590.000000,140.800000,85.532683,1687.000000,64.000000,71.400000,237.500000


In [140]:
df.head()

,country,code,year,health_exp,inci_tuberc,infant_mortality,life_expect,maternal_mortality,neonatal_mortality,prev_undernourishment,under_5_mortality
0,Afghanistan,AFG,1999,NaN,NaN,94.6,54.846,NaN,64.0,NaN,135.8
1,Afghanistan,AFG,2000,NaN,190.0,92.0,55.298,1346.0,62.7,NaN,131.6
2,Afghanistan,AFG,2001,NaN,189.0,89.3,55.798,1273.0,61.5,46.0,127.4
3,Afghanistan,AFG,2002,9.443391,189.0,86.6,56.454,1277.0,60.2,43.7,123.0
4,Afghanistan,AFG,2003,8.941258,189.0,83.7,57.344,1196.0,58.9,38.6,118.5


In [141]:
df['year'].unique()

array([1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023], dtype=int64)

Finding null values year-wise

This is to remove those years from data where most of the data is missing and to decide on the range of timeline from when
analysis needs to be done

In [142]:
null_prevalence = df.isnull().groupby(df['year']).sum()
total_rows = df.groupby('year').size()
null_percentage = (null_prevalence.div(total_rows, axis=0)) * 100

In [143]:
null_percentage

,country,code,year,health_exp,inci_tuberc,infant_mortality,life_expect,maternal_mortality,neonatal_mortality,prev_undernourishment,under_5_mortality
year,,,,,,,,,,,
1999,0.0,0.0,0.0,100.000000,100.000000,6.870229,1.526718,100.000000,6.870229,100.000000,6.870229
2000,0.0,0.0,0.0,13.207547,16.226415,7.924528,2.264151,12.075472,7.924528,100.000000,7.924528
2001,0.0,0.0,0.0,12.830189,16.226415,7.924528,2.641509,12.075472,7.924528,17.735849,7.924528
2002,0.0,0.0,0.0,12.452830,15.849057,7.924528,2.641509,12.075472,7.924528,17.735849,7.924528
2003,0.0,0.0,0.0,11.698113,15.849057,7.924528,2.641509,12.075472,7.924528,17.735849,7.924528
2004,0.0,0.0,0.0,11.698113,15.849057,7.924528,2.641509,12.075472,7.924528,17.735849,7.924528
2005,0.0,0.0,0.0,11.698113,15.094340,7.924528,2.264151,12.075472,7.924528,17.735849,7.924528
2006,0.0,0.0,0.0,11.698113,15.094340,7.924528,2.641509,12.075472,7.924528,17.735849,7.924528
2007,0.0,0.0,0.0,11.698113,15.094340,7.924528,2.641509,12.075472,7.924528,17.735849,7.924528


Year 1999 and 2023 needs to be removed as most data in these years is missing for most indicators

In [144]:
df = df[(df['year'] != 1999) & (df['year'] != 2023)]

Finding null values country wise

This is to remove those countries from data where most of the data is missing

In [145]:
null_prevalence_country = df.isnull().groupby(df['country']).sum()
total_rows_2 = df.groupby('country').size()
null_percentage_country = (null_prevalence_country.div(total_rows_2, axis=0)) * 100

In [58]:
pd.set_option('display.max_rows', None)  # Show all rows

In [146]:
null_percentage_country

,country,code,year,health_exp,inci_tuberc,infant_mortality,life_expect,maternal_mortality,neonatal_mortality,prev_undernourishment,under_5_mortality
country,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,13.043478,0.000000,0.0,0.000000,8.695652,0.0,4.347826,0.0
Africa Eastern and Southern,0.0,0.0,0.0,4.347826,100.000000,0.0,0.000000,8.695652,0.0,4.347826,0.0
Africa Western and Central,0.0,0.0,0.0,4.347826,100.000000,0.0,0.000000,8.695652,0.0,4.347826,0.0
Albania,0.0,0.0,0.0,4.347826,0.000000,0.0,0.000000,8.695652,0.0,4.347826,0.0
Algeria,0.0,0.0,0.0,4.347826,0.000000,0.0,0.000000,8.695652,0.0,4.347826,0.0
American Samoa,0.0,0.0,0.0,100.000000,0.000000,100.0,100.000000,100.000000,100.0,100.000000,100.0
Andorra,0.0,0.0,0.0,4.347826,0.000000,0.0,100.000000,100.000000,0.0,100.000000,0.0
Angola,0.0,0.0,0.0,4.347826,0.000000,0.0,0.000000,8.695652,0.0,4.347826,0.0
Antigua and Barbuda,0.0,0.0,0.0,4.347826,0.000000,0.0,0.000000,8.695652,0.0,100.000000,0.0


Now, a way has to be found to sort the data. Mean missing values will be calculated for each country as I have given equal importance to each indicator in the analysis.

In [147]:
mean_missing_by_country = null_percentage_country.mean(axis=1)

In [148]:
mean_missing_by_country.sort_values()

country
Slovenia                                                 1.185771
Korea, Rep.                                              1.185771
Iceland                                                  1.185771
Luxembourg                                               1.185771
Lithuania                                                1.185771
Norway                                                   1.185771
Sweden                                                   1.185771
North America                                            1.185771
Portugal                                                 1.185771
Poland                                                   1.185771
Denmark                                                  1.185771
Canada                                                   1.185771
Italy                                                    1.185771
Germany                                                  1.185771
Chile                                                    1.185771
Ir

In [149]:
threshold = 25

# Identify countries with mean missing data greater than the threshold
countries_to_remove = mean_missing_by_country[mean_missing_by_country > threshold].index

In [150]:
countries_to_remove

Index(['American Samoa', 'Andorra', 'Aruba', 'Bermuda',
       'British Virgin Islands', 'Cayman Islands', 'Channel Islands',
       'Curacao', 'Faroe Islands', 'French Polynesia', 'Gibraltar',
       'Greenland', 'Guam', 'Hong Kong SAR, China', 'Isle of Man', 'Kosovo',
       'Liechtenstein', 'Macao SAR, China', 'Monaco', 'New Caledonia',
       'Northern Mariana Islands', 'Palau', 'Puerto Rico', 'San Marino',
       'Sint Maarten (Dutch part)', 'St. Martin (French part)',
       'Turks and Caicos Islands', 'Virgin Islands (U.S.)'],
      dtype='object', name='country')

These countries will be removed as they will provide little to no value in our analysis. Moreover, all these countries are small island nations where data collection is challenge. For now, it makes sense to drop them.

In [151]:
df_cleaned = df[~df['country'].isin(countries_to_remove)]

dataset is now cleaned after dropping insignificant years, columns, and countries

In [152]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5451 entries, 1 to 6583
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                5451 non-null   object 
 1   code                   5451 non-null   object 
 2   year                   5451 non-null   int64  
 3   health_exp             5080 non-null   float64
 4   inci_tuberc            4761 non-null   float64
 5   infant_mortality       5451 non-null   float64
 6   life_expect            5451 non-null   float64
 7   maternal_mortality     4872 non-null   float64
 8   neonatal_mortality     5451 non-null   float64
 9   prev_undernourishment  4717 non-null   float64
 10  under_5_mortality      5451 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 511.0+ KB


For handling remaining null values of indicators, I will first see those countries which have 100% of values missing. These countries will be removed too.

In [153]:
columns_to_extract = ['country', 'year', 'health_exp']
df_health_exp = df_cleaned[columns_to_extract]

null_prevalence_country_health_exp = df_health_exp.isnull().groupby(df_health_exp['country']).sum()
total_rows = df_health_exp.groupby('country').size()
null_percentage_country = (null_prevalence_country_health_exp.div(total_rows, axis=0)) * 100
health_exp_null_percentage = null_percentage_country['health_exp']
countries_100_null_he = health_exp_null_percentage[health_exp_null_percentage == 100].index
countries_100_null_he

Index(['Korea, Dem. People's Rep.', 'Somalia', 'West Bank and Gaza'], dtype='object', name='country')

In [154]:
columns_to_extract = ['country', 'year', 'maternal_mortality']
df_mm = df_cleaned[columns_to_extract]

null_prevalence_country_mm = df_mm.isnull().groupby(df_mm['country']).sum()
total_rows = df_mm.groupby('country').size()
null_percentage_country = (null_prevalence_country_mm.div(total_rows, axis=0)) * 100

mm_null_percentage = null_percentage_country['maternal_mortality']
countries_100_null_mm = mm_null_percentage[mm_null_percentage == 100].index
countries_100_null_mm

Index(['Dominica', 'Marshall Islands', 'Nauru', 'St. Kitts and Nevis',
       'Tuvalu'],
      dtype='object', name='country')

In [155]:
columns_to_extract = ['country', 'year', 'inci_tuberc']
df_it = df_cleaned[columns_to_extract]

null_prevalence_country_it = df_it.isnull().groupby(df_it['country']).sum()
total_rows = df_it.groupby('country').size()
null_percentage_country = (null_prevalence_country_it.div(total_rows_2, axis=0)) * 100

it_null_percentage = null_percentage_country['inci_tuberc']
countries_100_null_it = it_null_percentage[it_null_percentage == 100].index
countries_100_null_it

Index(['Africa Eastern and Southern', 'Africa Western and Central',
       'Arab World', 'Caribbean small states',
       'Central Europe and the Baltics', 'Early-demographic dividend',
       'East Asia & Pacific (IDA & IBRD countries)', 'Euro area',
       'Europe & Central Asia (IDA & IBRD countries)', 'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'IBRD only',
       'IDA & IBRD total', 'IDA blend', 'IDA only', 'IDA total',
       'Late-demographic dividend',
       'Latin America & the Caribbean (IDA & IBRD countries)',
       'Least developed countries: UN classification',
       'Middle East & North Africa (IDA & IBRD countries)', 'OECD members',
       'Other small states', 'Pacific island small states',
       'Post-demographic dividend', 'Pre-demographic dividend', 'Small states',
       'South Asia (IDA & IBRD)', 'Sub-Saharan Africa (IDA & IBRD countries)'],
      dtype='object', name='country')

In [156]:
columns_to_extract = ['country', 'year', 'prev_undernourishment']
df_pu = df_cleaned[columns_to_extract]

null_prevalence_country_pu = df_pu.isnull().groupby(df_pu['country']).sum()
total_rows = df_pu.groupby('country').size()
null_percentage_country = (null_prevalence_country_pu.div(total_rows, axis=0)) * 100

pu_null_percentage = null_percentage_country['prev_undernourishment']
countries_100_null_pu = pu_null_percentage[pu_null_percentage == 100].index
countries_100_null_pu

Index(['Antigua and Barbuda', 'Bahamas, The', 'Bahrain', 'Bhutan',
       'Brunei Darussalam', 'Burundi', 'Equatorial Guinea', 'Eritrea',
       'Grenada', 'Lesotho', 'Maldives', 'Marshall Islands',
       'Micronesia, Fed. Sts.', 'Nauru', 'Qatar', 'Singapore',
       'St. Kitts and Nevis', 'St. Lucia', 'Tonga', 'Tuvalu',
       'West Bank and Gaza'],
      dtype='object', name='country')

While dealing with undernourishment data, it is seen that few of countries with 100 % data missing were significant in size. Dropping them entirely will distort our analysis.
So, these countries will not be dropped rather alternative source will be looked upon for this particular dimension.

Apart from undernourishment data, for rest 3 indicators, all the countries with 100 % data missing are removed

In [157]:
countries_to_remove = set(countries_100_null_he).union(countries_100_null_mm, countries_100_null_it)
countries_to_remove = list(countries_to_remove)
df_cleaned_2 = df_cleaned[~df_cleaned['country'].isin(countries_to_remove)]

In [158]:
df_cleaned_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4600 entries, 1 to 6583
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                4600 non-null   object 
 1   code                   4600 non-null   object 
 2   year                   4600 non-null   int64  
 3   health_exp             4332 non-null   float64
 4   inci_tuberc            4577 non-null   float64
 5   infant_mortality       4600 non-null   float64
 6   life_expect            4600 non-null   float64
 7   maternal_mortality     4200 non-null   float64
 8   neonatal_mortality     4600 non-null   float64
 9   prev_undernourishment  4017 non-null   float64
 10  under_5_mortality      4600 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 431.2+ KB


To deal with rest of null data with regards to health expenditure, maternal_mortality, and incident of tuberculosis.
First prevalance of nulls for specific indicator will be seen at country level, then suitable technique will be chosen

In [159]:
df_cleaned_2['country'].nunique()

200

In [160]:
df_cleaned_2.to_csv("stage_1.csv")

After viewing the above csv file in excel, it was found that for health expenditure all of the countries where missing data was there, it was only for 1 or 2 years and that too for initial years and latest years, so backfilling and forward filling are used.

In [161]:
df_cleaned_2 = df_cleaned_2.sort_values(by=['country', 'year'])

# Fill missing values for 'health_exp' using the next year's value for the same country
df_cleaned_2['health_exp'] = df_cleaned_2.groupby('country')['health_exp'].fillna(method='bfill')
df_cleaned_2['health_exp'] = df_cleaned_2.groupby('country')['health_exp'].fillna(method='ffill')

# Print the result to verify
clean = df_cleaned_2[['country', 'year', 'health_exp']]

In [162]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4600 entries, 1 to 6583
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     4600 non-null   object 
 1   year        4600 non-null   int64  
 2   health_exp  4600 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 143.8+ KB


In [163]:
df_cleaned_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4600 entries, 1 to 6583
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                4600 non-null   object 
 1   code                   4600 non-null   object 
 2   year                   4600 non-null   int64  
 3   health_exp             4600 non-null   float64
 4   inci_tuberc            4577 non-null   float64
 5   infant_mortality       4600 non-null   float64
 6   life_expect            4600 non-null   float64
 7   maternal_mortality     4200 non-null   float64
 8   neonatal_mortality     4600 non-null   float64
 9   prev_undernourishment  4017 non-null   float64
 10  under_5_mortality      4600 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 431.2+ KB


For handling incident of tuberculosis, null prevalance is seen at country level and problematic countries are identified

In [164]:
columns_to_extract = ['country', 'year', 'inci_tuberc']
df_it = df_cleaned_2[columns_to_extract]

null_prevalence_country_it = df_it.isnull().groupby(df_it['country']).sum()
total_rows = df_it.groupby('country').size()
null_percentage_country = (null_prevalence_country_it.div(total_rows, axis=0)) * 100
null_percentage_country.sort_values(by='inci_tuberc', ascending=False)

,country,year,inci_tuberc
country,,,
South Sudan,0.0,0.0,47.826087
Montenegro,0.0,0.0,21.739130
Serbia,0.0,0.0,21.739130
Timor-Leste,0.0,0.0,8.695652
Afghanistan,0.0,0.0,0.000000
Nepal,0.0,0.0,0.000000
Netherlands,0.0,0.0,0.000000
New Zealand,0.0,0.0,0.000000
Nicaragua,0.0,0.0,0.000000


In [165]:
df_cleaned_2.to_csv("stage_2_pre.csv")

After looking at csv file, values for tuberculosis incidents are also handeled by looking at data from alternative source 

In [166]:
df_3 = pd.read_csv("stage_2.csv")

In [167]:
df_3

,Unnamed: 0,country,code,year,health_exp,inci_tuberc,infant_mortality,life_expect,maternal_mortality,neonatal_mortality,prev_undernourishment,under_5_mortality
0,1,Afghanistan,AFG,2000,9.443391,190.00,92.000000,55.298000,1346.0,62.700000,NaN,131.600000
1,2,Afghanistan,AFG,2001,9.443391,189.00,89.300000,55.798000,1273.0,61.500000,46.000000,127.400000
2,3,Afghanistan,AFG,2002,9.443391,189.00,86.600000,56.454000,1277.0,60.200000,43.700000,123.000000
3,4,Afghanistan,AFG,2003,8.941258,189.00,83.700000,57.344000,1196.0,58.900000,38.600000,118.500000
4,5,Afghanistan,AFG,2004,9.808474,189.00,80.900000,57.944000,1115.0,57.500000,35.900000,114.000000
5,6,Afghanistan,AFG,2005,9.948289,189.00,78.000000,58.361000,1103.0,56.000000,34.200000,109.500000
6,7,Afghanistan,AFG,2006,10.622766,189.00,75.100000,58.684000,1044.0,54.600000,31.600000,105.000000
7,8,Afghanistan,AFG,2007,9.904675,189.00,72.300000,59.111000,1023.0,53.100000,28.300000,100.600000
8,9,Afghanistan,AFG,2008,10.256495,189.00,69.600000,59.852000,962.0,51.600000,25.100000,96.300000
9,10,Afghanistan,AFG,2009,9.818487,189.00,67.000000,60.364000,913.0,50.200000,21.600000,92.300000


Similar process for maternal mortality

In [168]:
columns_to_extract = ['country', 'year', 'maternal_mortality']
df_mm = df_3[columns_to_extract]

null_prevalence_country_mm = df_mm.isnull().groupby(df_mm['country']).sum()
total_rows = df_mm.groupby('country').size()
null_percentage_country = (null_prevalence_country_mm.div(total_rows, axis=0)) * 100
null_percentage_country.sort_values(by='maternal_mortality', ascending=False)

,country,year,maternal_mortality
country,,,
Afghanistan,0.0,0.0,8.695652
Oman,0.0,0.0,8.695652
Namibia,0.0,0.0,8.695652
Nepal,0.0,0.0,8.695652
Netherlands,0.0,0.0,8.695652
New Zealand,0.0,0.0,8.695652
Nicaragua,0.0,0.0,8.695652
Niger,0.0,0.0,8.695652
Nigeria,0.0,0.0,8.695652


As null prevalance is similar, it is highly probable that missing values is for same years and after looking at data it was
for initial and final years. So, backfilling and forward filling is used

In [169]:
df_3 = df_3.sort_values(by=['country', 'year'])

# Fill missing values for 'health_exp' using the next year's value for the same country
df_3['maternal_mortality'] = df_3.groupby('country')['maternal_mortality'].fillna(method='bfill')
df_3['maternal_mortality'] = df_3.groupby('country')['maternal_mortality'].fillna(method='ffill')

df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4600 entries, 0 to 4599
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             4600 non-null   int64  
 1   country                4600 non-null   object 
 2   code                   4600 non-null   object 
 3   year                   4600 non-null   int64  
 4   health_exp             4600 non-null   float64
 5   inci_tuberc            4600 non-null   float64
 6   infant_mortality       4600 non-null   float64
 7   life_expect            4600 non-null   float64
 8   maternal_mortality     4600 non-null   float64
 9   neonatal_mortality     4600 non-null   float64
 10  prev_undernourishment  4017 non-null   float64
 11  under_5_mortality      4600 non-null   float64
dtypes: float64(8), int64(2), object(2)
memory usage: 467.2+ KB


For undernourishment data, there is either 4.34 % missing data or 100 % missing data, for former backfilling and
forward filling is used and for latter alternative sources are looked

columns_to_extract = ['country', 'year', 'prev_undernourishment']
df_pu = df_3[columns_to_extract]

null_prevalence_country_pu = df_pu.isnull().groupby(df_pu['country']).sum()
total_rows = df_pu.groupby('country').size()
null_percentage_country = (null_prevalence_country_pu.div(total_rows, axis=0)) * 100
null_percentage_country.sort_values(by='prev_undernourishment', ascending=False)

In [170]:
df_3 = df_3.sort_values(by=['country', 'year'])

# Fill missing values for 'health_exp' using the next year's value for the same country
df_3['prev_undernourishment'] = df_3.groupby('country')['prev_undernourishment'].fillna(method='bfill')
df_3['prev_undernourishment'] = df_3.groupby('country')['prev_undernourishment'].fillna(method='ffill')

df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4600 entries, 0 to 4599
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             4600 non-null   int64  
 1   country                4600 non-null   object 
 2   code                   4600 non-null   object 
 3   year                   4600 non-null   int64  
 4   health_exp             4600 non-null   float64
 5   inci_tuberc            4600 non-null   float64
 6   infant_mortality       4600 non-null   float64
 7   life_expect            4600 non-null   float64
 8   maternal_mortality     4600 non-null   float64
 9   neonatal_mortality     4600 non-null   float64
 10  prev_undernourishment  4232 non-null   float64
 11  under_5_mortality      4600 non-null   float64
dtypes: float64(8), int64(2), object(2)
memory usage: 467.2+ KB


In [171]:
df_3.to_csv("stage_3_pre.csv", index = False)

In [172]:
df_final = pd.read_csv("stage_3.csv")

This data is cleaned and all the missing values are taken cared of, after filling data from alternative sources

In [173]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0.1           4600 non-null   int64  
 1   Unnamed: 0             4600 non-null   int64  
 2   country                4600 non-null   object 
 3   code                   4600 non-null   object 
 4   year                   4600 non-null   int64  
 5   health_exp             4600 non-null   float64
 6   inci_tuberc            4600 non-null   float64
 7   infant_mortality       4600 non-null   float64
 8   life_expect            4600 non-null   float64
 9   maternal_mortality     4600 non-null   int64  
 10  neonatal_mortality     4600 non-null   float64
 11  prev_undernourishment  4600 non-null   float64
 12  under_5_mortality      4600 non-null   float64
dtypes: float64(7), int64(4), object(2)
memory usage: 467.3+ KB


In the next step of data processing, More data related to region of countries and oecd classification will be added

In [174]:
url = "https://en.wikipedia.org/wiki/List_of_countries_and_territories_by_the_United_Nations_geoscheme"
tables = pd.read_html(url)


In [175]:
df_region = tables[0]

In [176]:
df_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Country or Area              248 non-null    object
 1   Geographical subregion       248 non-null    object
 2   Intermediary region[note 2]  248 non-null    object
 3   Continental region           248 non-null    object
 4   UNSD M49 Codes               248 non-null    object
dtypes: object(5)
memory usage: 9.8+ KB


In [177]:
columns_to_keep = ['Country or Area', 'Geographical subregion','Continental region']
df_region = df_region[columns_to_keep]

In [178]:
df_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Country or Area         248 non-null    object
 1   Geographical subregion  248 non-null    object
 2   Continental region      248 non-null    object
dtypes: object(3)
memory usage: 5.9+ KB


In [179]:
df_region = df_region.rename(columns={'Country or Area': 'country'})


In [180]:
df_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 248 non-null    object
 1   Geographical subregion  248 non-null    object
 2   Continental region      248 non-null    object
dtypes: object(3)
memory usage: 5.9+ KB


In [181]:
merged_df = pd.merge(df_final, df_region, on='country', how='left')

In [182]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4600 entries, 0 to 4599
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0.1            4600 non-null   int64  
 1   Unnamed: 0              4600 non-null   int64  
 2   country                 4600 non-null   object 
 3   code                    4600 non-null   object 
 4   year                    4600 non-null   int64  
 5   health_exp              4600 non-null   float64
 6   inci_tuberc             4600 non-null   float64
 7   infant_mortality        4600 non-null   float64
 8   life_expect             4600 non-null   float64
 9   maternal_mortality      4600 non-null   int64  
 10  neonatal_mortality      4600 non-null   float64
 11  prev_undernourishment   4600 non-null   float64
 12  under_5_mortality       4600 non-null   float64
 13  Geographical subregion  3496 non-null   object 
 14  Continental region      3496 non-null   

In [183]:
countries_with_null_region = merged_df[merged_df['Geographical subregion'].isnull()]
unique_countries = countries_with_null_region['country'].unique()
unique_countries

array(['Afghanistan', 'Bahamas, The', 'Bolivia', 'Cabo Verde',
       'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Czechia',
       'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)', 'Egypt, Arab Rep.',
       'Eswatini', 'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)', 'France',
       'Gambia, The', 'High income', 'Iran, Islamic Rep.', 'Korea, Rep.',
       'Kyrgyz Republic', 'Lao PDR', 'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Low & middle income', 'Low income', 'Lower middle income',
       'Micronesia, Fed. Sts.', 'Middle East & North Africa',
       'Middle East & North Africa (excluding high income)',
       'Middle income', 'Moldova', 'Myanmar', 'North America',
       'Slovak Republic', 'South Asia', 'St. Lucia',
       'St. Vincent and the Grenadines', 'Sub-Saharan Africa',
       'Sub-Saharan Africa (excluding high income)', 'Tanzania',
       'Timor-Leste

In [184]:
merged_df.to_csv("stage_4_pre.csv")

For all these countries, regions are mapped manually

In [185]:
#In this file all the missing regions are mapped manually

df4 = pd.read_csv("stage_4.csv")

Next... year-wise change in indicators per country will be calculated.

In [186]:
df4 = df4.sort_values(by=['country', 'year'])

In [187]:
indicators = ['health_exp', 'life_expect', 'infant_mortality','maternal_mortality','neonatal_mortality','under_5_mortality','inci_tuberc','prev_undernourishment']
for col in indicators:
    df4[f'{col}_change'] = df4.groupby('country')[col].diff()

This is final dataset with required structure, indicators, regions mapped, and year-wise changes.

In [188]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4600 entries, 0 to 4599
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0.1                  4600 non-null   int64  
 1   Unnamed: 0                    4600 non-null   int64  
 2   country                       4600 non-null   object 
 3   country_code                  4600 non-null   object 
 4   year                          4600 non-null   int64  
 5   health_exp                    4600 non-null   float64
 6   life_expect                   4600 non-null   float64
 7   maternal_mortality            4600 non-null   int64  
 8   infant_mortality              4600 non-null   float64
 9   neonatal_mortality            4600 non-null   float64
 10  under_5_mortality             4600 non-null   float64
 11  inci_tuberc                   4600 non-null   float64
 12  prev_undernourishment         4600 non-null   float64
 13  Geo

Note that, you will see nulls still existing for region columns, it was inspected and was found that country corresponding to these nulls are not exactly countries but rather, region itself. So, these nulls make sense.

In [190]:
df4.drop(df4.columns[[0,1]], axis=1, inplace=True)

In [191]:
df4.to_csv("merged.csv")

In [192]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4600 entries, 0 to 4599
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       4600 non-null   object 
 1   country_code                  4600 non-null   object 
 2   year                          4600 non-null   int64  
 3   health_exp                    4600 non-null   float64
 4   life_expect                   4600 non-null   float64
 5   maternal_mortality            4600 non-null   int64  
 6   infant_mortality              4600 non-null   float64
 7   neonatal_mortality            4600 non-null   float64
 8   under_5_mortality             4600 non-null   float64
 9   inci_tuberc                   4600 non-null   float64
 10  prev_undernourishment         4600 non-null   float64
 11  Geographical subregion        4163 non-null   object 
 12  Continental region            4163 non-null   object 
 13  hea